In [3]:
import pandas as pd
import plotly.graph_objects as go
import os
import numpy as np
import matplotlib.pyplot as plt


In [6]:
# Sankey of food GWP flows (Python)
# Requires: pandas, plotly (pip install pandas plotly)


# 1) Load your Excel file
path = "Hospital_GWP_Data.xlsx"  # change if needed
df = pd.read_excel(path)

# 2) Define categories
animal = ["Beef", "All Other Animal Meat/Fish", "Dairy & Alternatives"]
all_cats = df["Category"].tolist()
non_animal = [c for c in all_cats if c not in animal]

# 3) Helper to compute values by month or total
def make_sankey(month="Total"):
    months = ["Dec","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov"]
    d = df.copy()

    if month == "Total":
        d["Value"] = d[months].sum(axis=1)
    else:
        if month not in months:
            raise ValueError(f"month must be one of {months} or 'Total'")
        d["Value"] = d[month]

    total_val = d["Value"].sum()
    animal_val = d.loc[d["Category"].isin(animal), "Value"].sum()
    non_animal_val = total_val - animal_val

    # 4) Build nodes
    nodes = ["Total", "Animal-based", "Non-animal-based"] + animal + non_animal
    node_index = {n:i for i,n in enumerate(nodes)}

    # 5) Build links
    sources = []
    targets = []
    values  = []

    # Total -> Animal vs Non-animal
    sources += [node_index["Total"], node_index["Total"]]
    targets += [node_index["Animal-based"], node_index["Non-animal-based"]]
    values  += [animal_val, non_animal_val]

    # Animal-based -> specific animal categories
    for c in animal:
        v = float(d.loc[d["Category"] == c, "Value"])
        sources.append(node_index["Animal-based"])
        targets.append(node_index[c])
        values.append(v)

    # Non-animal-based -> specific non-animal categories
    for c in non_animal:
        v = float(d.loc[d["Category"] == c, "Value"])
        sources.append(node_index["Non-animal-based"])
        targets.append(node_index[c])
        values.append(v)

    # 6) Plot
    fig = go.Figure(data=[go.Sankey(
        arrangement="snap",
        node=dict(
            label=nodes,
            pad=18,
            thickness=20,
            line=dict(width=0.5, color="gray")
        ),
        link=dict(source=sources, target=targets, value=values)
    )])

    title = f"GWP flows (kg CO2e) — {month}" if month != "Total" else "GWP flows (kg CO2e) — Total year"
    fig.update_layout(title=title, font=dict(size=13), height=650, width=1000)
    return fig

# 7) Choose one view to render
# Options: "Total" or any month like "Dec","Jan",...,"Nov"
fig = make_sankey(month="Total")
fig.show()

# Optional: save to HTML for easy sharing
# fig.write_html("GWP_Sankey_Total.html", include_plotlyjs="cdn")


In [12]:
df

,Category,Dec,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Total_GWP
0,Beef,80388.505901,84655.555105,81092.159182,75244.853287,91835.545254,78204.314925,74112.441721,79436.779365,106363.662238,99245.773873,79749.291569,100801.212108,1.031130e+06
1,All Other Animal Meat/Fish,19568.400978,17887.749488,18587.171684,18364.760594,17915.860577,17705.367971,25080.817221,18418.359405,14294.101264,17567.203593,17755.836409,18128.368707,2.212740e+05
2,Dairy & Alternatives,56331.135399,60181.438215,52825.793721,56308.439090,62080.504124,57911.229862,57295.996941,58927.472364,56349.095217,57467.849181,56914.736674,58202.895917,6.907966e+05
3,Vegetables & Fruits,22860.080682,24143.049895,24536.931078,24301.482214,24855.060380,24072.336579,23679.870151,23934.108227,23579.566732,23661.924833,23597.972807,24667.498463,2.878899e+05
4,Grains & Starches,13994.522801,14562.885925,14806.493039,17283.349754,14883.036084,21486.937352,14667.017268,14684.315441,14734.996609,14900.976482,18266.956830,15390.024769,1.896615e+05
5,Plant Based Proteins,323.237585,355.424730,401.652138,381.465588,432.801021,452.872339,386.407438,384.337009,389.522543,422.088502,483.335801,461.633059,4.874778e+03
6,Supplement Drinks,29081.961310,42324.555880,33129.699310,49075.816620,37663.760860,36740.602440,37310.533320,35316.751190,26069.796640,46979.296600,46815.653830,52295.997730,4.728044e+05
7,Beverages,7081.479703,7334.354806,5570.744764,4887.947193,5991.948486,5208.874838,5272.534479,5291.603562,5246.362083,5197.811035,5285.851890,5430.644920,6.780016e+04
8,Desserts,20965.645831,54472.771324,20451.362062,19562.878340,21709.390821,22244.442162,21901.200942,20378.617350,21952.161826,21561.795759,21418.279297,22884.858361,2.895034e+05


In [11]:
import pandas as pd
import plotly.graph_objects as go

# 1. load your excel file
path = "Hospital_GWP_Data.xlsx"
df = pd.read_excel(path)

# 2. calculate total GWP for each category
months = ["Dec","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov"]
df["Total_GWP"] = df[months].sum(axis=1)

# 3. define categories
animal = ["Beef", "All Other Animal Meat/Fish", "Dairy & Alternatives"]
non_animal = [c for c in df["Category"] if c not in animal]

# 4. compute group sums
animal_total = df.loc[df["Category"].isin(animal), "Total_GWP"].sum()
non_animal_total = df["Total_GWP"].sum() - animal_total

# 5. build nodes and links
nodes = ["Total", "Animal-based", "Non-animal-based"] + animal + non_animal
node_index = {n:i for i,n in enumerate(nodes)}

sources = []
targets = []
values  = []

# total -> animal/non-animal
sources += [node_index["Total"], node_index["Total"]]
targets += [node_index["Animal-based"], node_index["Non-animal-based"]]
values  += [animal_total, non_animal_total]

# animal -> each animal category
for c in animal:
    v = df.loc[df["Category"] == c, "Total_GWP"].iloc[0]
    sources.append(node_index["Animal-based"])
    targets.append(node_index[c])
    values.append(v)

# non-animal -> each non-animal category
for c in non_animal:
    v = df.loc[df["Category"] == c, "Total_GWP"].iloc[0]
    sources.append(node_index["Non-animal-based"])
    targets.append(node_index[c])
    values.append(v)

# 6. make the sankey plot
fig = go.Figure(data=[go.Sankey(
    arrangement="snap",
    node=dict(
        label=nodes,
        pad=18,
        thickness=20,
        line=dict(width=0.5, color="gray")
    ),
    link=dict(source=sources, target=targets, value=values)
)])

fig.update_layout(
    title="Hospital Food GWP Flows (kg CO₂e, Total Year)",
    font=dict(size=13),
    height=650,
    width=1000
)

fig.show()
